In [ ]:
import os
import sys
sys.path.insert(1,os.path.join(os.path.realpath('__file__').split("lacrosse-prediction")[0],'lacrosse-prediction'))


#### Generate Data Folder for given Category

In [ ]:
from etl.etl_helper import create_data_folders

In [ ]:
from config.config import category_config_dict
from config.config import DATA_FOLDER_PATH

In [ ]:
testing_category_dict = category_config_dict['testing']

In [ ]:
create_data_folders(
    testing_category_dict['category_name'],
    DATA_FOLDER_PATH,
    testing_category_dict['websources_list']
    )

In [ ]:
def gen_data_folder_paths(
        category_name:str,
        path:str,
        data_layers=['raw','bronze','silver','gold']
        ):
    return os.path.join(path,category_name,data_layers[0]),os.path.join(path,category_name,data_layers[1]),os.path.join(path,category_name,data_layers[2]),os.path.join(path,category_name,data_layers[3])

#### Put data in raw folder using scraper

In [ ]:
gen_data_folder_paths('testing',DATA_FOLDER_PATH)

In [ ]:
# we do it mannual here

#### Run ETL to process data

In [ ]:
import pandas as pd

In [ ]:
from etl.etl_helper import process_raw_to_bronze, process_bronze_to_silver, process_silver_to_gold

In [ ]:
CATEGORY_NAME = testing_category_dict['category_name']
print(CATEGORY_NAME)
SOURCE_FOLDERS = testing_category_dict['websources_list']
SOURCE_FOLDERS

In [ ]:
process_raw_to_bronze(os.path.join(DATA_FOLDER_PATH,CATEGORY_NAME,'raw'),os.path.join(DATA_FOLDER_PATH,CATEGORY_NAME,'bronze'))

In [ ]:
# add some functionality there


In [ ]:
# now process from bronze to silver layer
process_bronze_to_silver(os.path.join(DATA_FOLDER_PATH,CATEGORY_NAME,'bronze'),os.path.join(DATA_FOLDER_PATH,CATEGORY_NAME,'silver'))

In [ ]:
# now process from bronze to silver layer
process_silver_to_gold(os.path.join(DATA_FOLDER_PATH,CATEGORY_NAME,'silver'),os.path.join(DATA_FOLDER_PATH,CATEGORY_NAME,'gold'),CATEGORY_NAME+'.csv')

In [ ]:
# move file from gold stage to final stage folder

#### Create DB tables 

In [ ]:
# get connction from db connector
# get schema for table
# execute query to create schema in database


In [ ]:
CSV_FILE_PATH = '../data/final_stage/testing.csv'
TARGET_TABLE_NAME = 'testingTable'
unique_index_cols=[]

In [ ]:
from etl.db_and_schema_helper import gen_table_schema_create_query
from etl.db_and_schema_helper import query_column_types_map
from etl.db_and_schema_helper import dev_db_cnn
from etl.db_and_schema_helper import engine
from etl.db_and_schema_helper import table_exist
from etl.db_and_schema_helper import load_df_to_database

In [ ]:
schema_query=gen_table_schema_create_query(
    pd.read_csv(CSV_FILE_PATH),
    TARGET_TABLE_NAME,
    query_column_types_map,
    unique_index_columns=['event','event_date','gender','state','web_source']
)

print(schema_query)

In [ ]:
if not table_exist(TARGET_TABLE_NAME,engine):
    dev_db_cnn._execute_query(schema_query)
    print("table create success")


In [ ]:
load_df_to_database(pd.read_csv(CSV_FILE_PATH),TARGET_TABLE_NAME,engine,unique_index_cols)